In [13]:
from PIL import Image
import os
from transformers import CLIPProcessor, CLIPModel
import cv2
from src.data import utils
import torch
import pandas as pd
from datetime import datetime
import torchvision.transforms as transforms
from tqdm import tqdm

In [12]:
def add_to_database(
   now: str, features, prediction: int):
   with open('../app/referrence_database.csv', 'a+') as file:
      # Convert features to list
      features = features.tolist()[0]
      entry = f"{now},"
      for feature in features:
         entry += str(feature) + ", "
      entry += str(prediction) + "\n"

      if os.stat('../app/referrence_database.csv').st_size == 0:
         # Write header
         header = "timestamp,"
         for i in range(1, len(features)):
            header += f"feature_{i}, "
         header += "prediction\n"
         file.write(header)
         file.write(entry)
      else:
         file.write(entry)
   return None

In [22]:
def predict_step(image_paths):
   # Load model checkpoint
   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   # Get latest model checkpoint from models/trained_models
   latest_model_checkpoint = max([os.path.join("../models/trained_models", f) for f in os.listdir("../models/trained_models")], key=os.path.getctime)

   model = torch.load(latest_model_checkpoint)

   # Load image
   image = Image.open(image_paths[0])
   transform = transforms.ToTensor()
   image = transform(image).unsqueeze(dim=0)
   with torch.no_grad():
      log_ps = model(image)

      ps = torch.exp(log_ps)
      top_p, top_class = ps.topk(1, dim=1)

   return top_class.item()

In [ ]:
# Loop over each image in data/raw/train
data = pd.read_csv("../data/raw/BUTTERFLIES.csv")
trainset = data[data["data set"] == "train"]
h, w = 224, 224

for i, row in tqdm(trainset.iterrows()):
    img_path = f"../data/raw/{row['filepaths']}"

    # Create metadata for image
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    # Load image as PIL
    img = Image.open(img_path)

    inputs = processor(text=None, images=img, return_tensors="pt", padding=True)
    img_features = model.get_image_features(inputs['pixel_values'])

    # Predict
    preds = predict_step([img_path])
    add_to_database(now, img_features, preds)